# Preprocessing
- Stem the Data (run/running/ran --> ran)
- Identify the most commonly used words in decription
- One hot encody top top 10(ish) of the nontrvial words
- Scale the Data
- PCA (95% of variation captured)

# Models

- Linear Regression
- Decision Tree Regression
- [Plot Tree Regression](https://scikit-learn.org/stable/auto_examples/tree/plot_tree_regression.html)
- Perceptron
- Neural Netwrok

# Set Hyperparameters

Use 10-fold cross validation to tune hyperparameters
- using RSS as an error metric?

In [ ]:
print('hello world')